In [178]:
# Import PyTorch
import torch
# Get pandas for data manipulation
import pandas as pd
# Import nltk for text processing
import nltk
# Import os for file manipulation
import os
# Import train_test_split from sklearn
from sklearn.model_selection import train_test_split
# Import LabelEncoder from sklearn for encoding labels
from sklearn.preprocessing import LabelEncoder

In [179]:
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [180]:
# Load the data from Datasets/goodreads_data_onehot_genres.csv
data = pd.read_csv('Datasets/goodreads_data_onehot_genres.csv')
# Show the first few rows of the data
print(data.head())
print(data.shape)

                                                Book  \
0                              To Kill a Mockingbird   
1  Harry Potter and the Philosopher’s Stone (Harr...   
2                                Pride and Prejudice   
3                          The Diary of a Young Girl   
4                                  The Little Prince   

                                         Description  Fantasy  Adult  \
0  The unforgettable novel of a childhood in a sl...        0      1   
1  Harry Potter thinks he is an ordinary boy - un...        1      1   
2  Since its immediate success in 1813, Pride and...        0      0   
3  Discovered in the attic in which she spent the...        0      0   
4  A pilot stranded in the desert awakes one morn...        1      1   

   Historical  Roman  Romance  Young Adult  Historical Fiction  Science  ...  \
0           1      0        0            1                   1        0  ...   
1           0      0        0            1                   0        

In [181]:
# Create a dataframe with the plot column tokenized and lowercased
tokenizeddf = data.copy()
tokenizeddf['Description'] = tokenizeddf['Description'].apply(lambda x: nltk.word_tokenize(x.lower()))

In [182]:
# Get the length of the longest plot
maxlen = tokenizeddf['Description'].apply(len).max()

# Get the set of all words in the plot column
wordset = set()
for plot in tokenizeddf['Description']:
    wordset.update(plot)
# Get the number of unique words
numwords = len(wordset)
print("Number of unique words: ", numwords)

# Create a dictionary that maps words to integers
word2int = {word: i for i, word in enumerate(wordset)}

# Function to convert a list of words to a list of integers
def words2ints(words):
    # Run through each word in the list
    ints = []
    for word in words:
        # If the word is in the dictionary, add the integer to the list
        if word in word2int:
            ints.append(word2int[word])
    return ints

Number of unique words:  80123


In [183]:
# Convert the Description column to a list of integers
tokenizeddf['Description'] = tokenizeddf['Description'].apply(words2ints)

# Pad the sequences to the maximum length
def pad_description(description):
    return description + [0] * (maxlen - len(description))

tokenizeddf['Description'] = tokenizeddf['Description'].apply(pad_description)

In [184]:
# Create a label encoder
le = LabelEncoder()

In [185]:
# Dataset class for genres
class GenreDataset(torch.utils.data.Dataset):
    def __init__(self, data, genre):
        self.plot = data["Description"].values
        self.genre = data[genre].values

    def __len__(self):
        return len(self.plot)
    
    def __getitem__(self, i):
        plot = self.plot[i]
        genre = self.genre[i]
        return torch.tensor(plot, dtype=torch.long), torch.tensor(genre, dtype=torch.float)
    
# Dataloader class for genres
class GenreDataLoader(torch.utils.data.DataLoader):
    def __init__(self, dataset, batch_size=32, shuffle=True):
        super().__init__(dataset, batch_size=batch_size, shuffle=shuffle)

# RNN class for genres
# Create the RNN to classify the plots as action or not
class GenreRNN(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(GenreRNN, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        self.sigmoid = torch.nn.Sigmoid()
        self.hidden_dim = hidden_dim
    
    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])        
        return out

In [197]:
# Function to train a model
def train_genre_rnn(genre, epochs, embed, hidden, output):
    genremodel = GenreRNN(numwords, embed, hidden, output)
    # Extract the relevant columns from the dataframe
    genredf = tokenizeddf[['Book', 'Description', genre]]
    # Encode the genre column
    genredf[genre] = le.fit_transform(genredf[genre])
    # Train test split the data
    # We won't use the test data in this function
    genretrain, genretest = train_test_split(genredf, test_size=0.2, random_state=42)
    # Create a GenreDataset object
    genretraindataset = GenreDataset(genretrain, genre)
    # Create a DataLoader
    genretrainloader = GenreDataLoader(genretraindataset, batch_size=32, shuffle=True)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(genremodel.parameters(), lr=0.01)

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Send the model to the device
    genremodel.to(device)

    # Train the model
    num_epochs = epochs
    for epoch in range(num_epochs):
        epoch_loss = 0
        genremodel.train()
        for plots, genres in genretrainloader:
            # Send the data to the device
            plots = plots.to(device)
            genres = genres.to(device)
            outputs = genremodel(plots)
            loss = criterion(outputs, genres.long())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # Return the model
    return genremodel

In [198]:
# Save a model
def save_model(model, genre):
    if not os.path.exists("models/rnn"):
        os.makedirs("models/rnn")
    torch.save(model.state_dict(), f"models/rnn/{genre}modelgoodreads.pth")

In [188]:
# Get all the genres
genres = data.columns[2:]
print(genres)

Index(['Fantasy', 'Adult', 'Historical', 'Roman', 'Romance', 'Young Adult',
       'Historical Fiction', 'Science', 'Mystery', 'Contemporary', 'Thriller',
       'Science Fiction', 'History', 'Adventure', 'Philosophy', 'Biography',
       'Crime', 'Self Help', 'Psychology', 'Mystery Thriller', 'Memoir',
       'Childrens', 'Humor', 'Suspense', 'Horror'],
      dtype='object')


In [199]:
# Train a model for every genre
for genre in genres:
    print(f"Training model for {genre}")
    model = train_genre_rnn(genre, 10, 128, 128, 2)
    save_model(model, genre)

Training model for Fantasy


C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\1723805196.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/10, Loss: 0.45378658175468445
Epoch 2/10, Loss: 0.6822710633277893
Epoch 3/10, Loss: 0.7687479853630066
Epoch 4/10, Loss: 0.6806064248085022
Epoch 5/10, Loss: 0.4537822902202606
Epoch 6/10, Loss: 0.637323796749115
Epoch 7/10, Loss: 0.49676191806793213
Epoch 8/10, Loss: 0.37705138325691223
Epoch 9/10, Loss: 0.5785762071609497
Epoch 10/10, Loss: 0.7937762141227722
Training model for Adult


C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\1723805196.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/10, Loss: 0.5647515058517456
Epoch 2/10, Loss: 0.4574686288833618
Epoch 3/10, Loss: 0.5970885157585144
Epoch 4/10, Loss: 0.5634920001029968
Epoch 5/10, Loss: 0.6033783555030823
Epoch 6/10, Loss: 0.46666255593299866
Epoch 7/10, Loss: 0.4614276587963104
Epoch 8/10, Loss: 0.9936614036560059
Epoch 9/10, Loss: 0.8872262835502625
Epoch 10/10, Loss: 0.5059335231781006
Training model for Historical


...

Epoch 1/10, Loss: 0.3057411313056946
Epoch 2/10, Loss: 0.30704060196876526
Epoch 3/10, Loss: 0.5095469951629639
Epoch 4/10, Loss: 0.12641550600528717
Epoch 5/10, Loss: 0.10002268105745316
Epoch 6/10, Loss: 0.07427483052015305
Epoch 7/10, Loss: 0.29992905259132385
Epoch 8/10, Loss: 0.2993635833263397
Epoch 9/10, Loss: 1.0793366432189941
Epoch 10/10, Loss: 0.12552405893802643
Training model for Suspense


C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\1723805196.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/10, Loss: 0.3958897888660431
Epoch 2/10, Loss: 0.015355576761066914
Epoch 3/10, Loss: 0.2890794277191162
Epoch 4/10, Loss: 0.2905731499195099
Epoch 5/10, Loss: 0.3008432984352112
Epoch 6/10, Loss: 0.36601749062538147
Epoch 7/10, Loss: 0.31811654567718506
Epoch 8/10, Loss: 0.05802286043763161
Epoch 9/10, Loss: 0.3655276596546173
Epoch 10/10, Loss: 0.33874285221099854
Training model for Horror


C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\1723805196.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])


Epoch 1/10, Loss: 0.299934059381485
Epoch 2/10, Loss: 0.7591667175292969
Epoch 3/10, Loss: 0.06789998710155487
Epoch 4/10, Loss: 0.017646221444010735
Epoch 5/10, Loss: 0.09602639824151993
Epoch 6/10, Loss: 0.09561198204755783
Epoch 7/10, Loss: 0.28840336203575134
Epoch 8/10, Loss: 0.08470696955919266
Epoch 9/10, Loss: 0.3415755033493042
Epoch 10/10, Loss: 0.13521063327789307


In [200]:
# Test the models
# Returns the accuracy, precision, recall, and F1 score of the model on the test data
def test_models(genres):
    # Create a dictionary to store the metrics
    results = {}
    # Run through each genre
    for genre in genres:
        # Load the model from the file (if it exists)
        if os.path.exists(f"models/rnn/{genre}modelgoodreads.pth"):
            model = GenreRNN(numwords, 128, 128, 2)
            model.load_state_dict(torch.load(f"models/rnn/{genre}modelgoodreads.pth"))
            model.eval()
            # Extract the relevant columns from the dataframe
            genredf = tokenizeddf[['Book', 'Description', genre]]
            # Encode the genre column
            genredf[genre] = le.fit_transform(genredf[genre])
            # Train test split the data
            genretrain, genretest = train_test_split(genredf, test_size=0.3, random_state=42)
            genretestdataset = GenreDataset(genretest, genre)
            genretestloader = GenreDataLoader(genretestdataset, batch_size=32, shuffle=True)
            # Set the device
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            # Send the model to the device
            model.to(device)
            # Initialize the metrics
            tp = 0
            tn = 0
            fp = 0
            fn = 0
            # Turn off gradients
            with torch.no_grad():
                for plots, genres in genretestloader:
                    plots = plots.to(device)
                    genres = genres.to(device)
                    # Get the outputs
                    outputs = model(plots)
                    _, preds = torch.max(outputs, 1)
                    # Increment the metrics
                    newtp = torch.sum((preds == 1) & (genres == 1)).item()
                    newtn = torch.sum((preds == 0) & (genres == 0)).item()
                    newfp = torch.sum((preds == 1) & (genres == 0)).item()
                    newfn = torch.sum((preds == 0) & (genres == 1)).item()
                    tp += newtp
                    tn += newtn
                    fp += newfp
                    fn += newfn
            # Calculate the metrics
            accuracy = (tp + tn) / (tp + tn + fp + fn)
            precision = tp / (tp + fp) if tp + fp != 0 else 0
            recall = tp / (tp + fn) if tp + fn != 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0                    
            results[genre] = { "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1": f1 }
    return results

In [191]:
# Write the results to a file
results = test_models(genres)
with open("goodreads_rnn_results.txt", "w") as f:
    for genre, metrics in results.items():
        f.write(f"{genre}:\n")
        for metric, value in metrics.items():
            f.write(f"{metric}: {value}\n")
        f.write("\n")

C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\4076232601.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\4076232601.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genredf[genre] = le.fit_transform(genredf[genre])
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\4076232601.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [192]:
# Predict the probability that a plot is a certain genre
def score_genre(description,genre):
    # Get the tokenized version of the plot
    tokenplot = nltk.word_tokenize(description.lower())
    tokenplot = words2ints(tokenplot)
    tokenplot = pad_description(tokenplot)
    # Check if the model exists and load it
    if not os.path.exists(f"models/rnn/{genre}modelgoodreads.pth"):
        print("Model does not exist")
        return None
    model = GenreRNN(numwords, 128, 128, 2)
    model.load_state_dict(torch.load(f"models/rnn/{genre}modelgoodreads.pth"))
    model.eval()
    # Convert the plot to a tensor
    tokenplot = torch.tensor(tokenplot, dtype=torch.long).unsqueeze(0)
    # Get the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Send the plot and the model to the device
    tokenplot = tokenplot.to(device)
    model.to(device)
    # Get the output of the model
    output = model(tokenplot)
    # Turn the output into a probability that the plot is the genre
    prob = torch.nn.functional.softmax(output, dim=1)[:, 1]
    return prob.item()

# Test the function
print(score_genre(data['Description'][0], 'Fantasy'))

0.24978426098823547


In [193]:
# Function to get the genre score for a plot
def predict_genres(description, genre):
    # Tokenize and pad the plot
    description = nltk.word_tokenize(description.lower())
    description = words2ints(description)
    description = pad_description(description)
    description = torch.tensor(description, dtype=torch.long).unsqueeze(0)
    # Check if the model exists
    if not os.path.exists(f"models/rnn/{genre}modelgoodreads.pth"):
        print(f"Model for {genre} does not exist")
        return None
    #print(f"Predicting {genre}")
    # Load the model
    model = GenreRNN(numwords, 128, 128, 2)
    model.load_state_dict(torch.load(f"models/rnn/{genre}modelgoodreads.pth"))
    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Send the model to the device
    model.to(device)
    # Send the plot to the device
    description = description.to(device)
    # Turn off gradients
    with torch.no_grad():
        # Get the output
        output = model(description)
        _, preds = torch.max(output, 1)
        return preds.item()

In [194]:
# Create a new dataframe with the Book and Description columns
genre_scores = data[['Book', 'Description']]
# Add a new column for the genre scores
for genre in genres:
    genre_scores[genre] = 0

C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\4115772836.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_scores[genre] = 0
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\4115772836.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_scores[genre] = 0
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\4115772836.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [195]:
# Predict genres for the first book
row = genre_scores.iloc[0]
for genre in genres:
    # Predict the genre score for the plot
    score = score_genre(row['Description'], genre)
    # Print the score and genre
    print(f"{genre}: {score}")
    # Overwrite the genre score in the dataframe
    genre_scores.at[0, genre] = score

row = genre_scores.iloc[1]
for genre in genres:
    # Predict the genre score for the plot
    score = score_genre(row['Description'], genre)
    # Print the score and genre
    print(f"{genre}: {score}")
    # Overwrite the genre score in the dataframe
    genre_scores.at[1, genre] = score

print(data['Description'][0])
print(data['Description'][1])

print(score_genre(data['Description'][0], 'Fantasy'))
print(score_genre(data['Description'][1], 'Fantasy'))

Fantasy: 0.24978426098823547
Adult: 0.2589132487773895


C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.24978426098823547' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0, genre] = score
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2589132487773895' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0, genre] = score
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.21866098046302795' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0,

Historical: 0.21866098046302795
Roman: 0.174833282828331
Romance: 0.1999729573726654
Young Adult: 0.21275590360164642


C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.174833282828331' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0, genre] = score
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1999729573726654' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0, genre] = score
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.21275590360164642' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0, g

Historical Fiction: 0.17600159347057343
Science: 0.1967347264289856


...

Childrens: 0.06173611432313919


C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.06173611432313919' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0, genre] = score


Humor: 0.07380232959985733
Suspense: 0.07185681164264679
Horror: 0.05504041910171509


C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.07380232959985733' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0, genre] = score
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.07185681164264679' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0, genre] = score
C:\Users\zkand\AppData\Local\Temp\ipykernel_48300\810474042.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.05504041910171509' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  genre_scores.at[0

Fantasy: 0.24978426098823547
Adult: 0.25891321897506714
Historical: 0.21866096556186676
Roman: 0.1748332679271698
Romance: 0.1999729573726654
Young Adult: 0.21275590360164642
Historical Fiction: 0.17600159347057343
Science: 0.1967347264289856
Mystery: 0.1720680594444275
Contemporary: 0.1538202315568924
Thriller: 0.1474837064743042
Science Fiction: 0.11311233043670654
History: 0.10035469383001328
Adventure: 0.08986809104681015
Philosophy: 0.0732497125864029
Biography: 0.09064339101314545
Crime: 0.0889410600066185
Self Help: 0.09301909804344177
Psychology: 0.08704997599124908
Mystery Thriller: 0.08307089656591415
Memoir: 0.07320615649223328
Childrens: 0.061736129224300385
Humor: 0.07380231469869614
Suspense: 0.07185681164264679
Horror: 0.05504041910171509
The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went 

In [196]:
print(genre_scores.head())

                                                Book  \
0                              To Kill a Mockingbird   
1  Harry Potter and the Philosopher’s Stone (Harr...   
2                                Pride and Prejudice   
3                          The Diary of a Young Girl   
4                                  The Little Prince   

                                         Description   Fantasy     Adult  \
0  The unforgettable novel of a childhood in a sl...  0.249784  0.258913   
1  Harry Potter thinks he is an ordinary boy - un...  0.249784  0.258913   
2  Since its immediate success in 1813, Pride and...  0.000000  0.000000   
3  Discovered in the attic in which she spent the...  0.000000  0.000000   
4  A pilot stranded in the desert awakes one morn...  0.000000  0.000000   

   Historical     Roman   Romance  Young Adult  Historical Fiction   Science  \
0    0.218661  0.174833  0.199973     0.212756            0.176002  0.196735   
1    0.218661  0.174833  0.199973     0.212756

In [131]:
# Score each plot for each genre
for genre in genres:
    print(f"Scoring {genre}")
    genre_scores[genre] = genre_scores['Description'].apply(lambda x: score_genre(x, genre))

Scoring Fantasy
Scoring Adult
Scoring Historical
Scoring Roman
Scoring Romance
Scoring Young Adult
Scoring Historical Fiction
Scoring Science
Scoring Mystery
Scoring Contemporary
Scoring Thriller
Scoring Science Fiction
Scoring History
Scoring Adventure
Scoring Philosophy
Scoring Biography
Scoring Crime
Scoring Self Help
Scoring Psychology
Scoring Mystery Thriller
Scoring Memoir
Scoring Childrens
Scoring Humor
Scoring Suspense
Scoring Horror


In [132]:
# Save the dataframe to a CSV file
genre_scores.to_csv('Datasets/goodreads_rnn_genre_scores.csv', index=False)